In [1]:
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
import logging
import pennylane as qml
import numpy as np
import random
import torch

In [2]:
config = {
    "qubits": 4,  
    "layers": 3,  
}

#dev = qml.device('default.qubit', wires=config["qubits"])
dev = qml.device('lightning.kokkos', wires=config["qubits"])
#dev = qml.device('lightning.gpu', wires=config["qubits"], mpi = True)
#dev = qml.device('lightning.gpu', wires=config["qubits"])

@qml.qnode(dev, interface='torch', diff_method='adjoint')
def gen_circuit(w):
    z1 = random.uniform(-1, 1)
    z2 = random.uniform(-1, 1)
    for i in range(config["qubits"]):
        qml.RY(np.arcsin(z1), wires=i)
        qml.RZ(np.arcsin(z2), wires=i)
    for l in range(config["layers"]):
        for i in range(config["qubits"]):
            qml.RY(w[i], wires=i)
        for i in range(config["qubits"]-1):
            qml.CNOT(wires=[i, i+1])
            qml.RZ(w[i + config["qubits"]], wires=i+1)
            qml.CNOT(wires=[i, i+1])
    #return qml.expval(qml.PauliZ(1)) 
    return [qml.expval(qml.PauliZ(i)) for i in range(config["qubits"])]


w = np.random.rand(config["qubits"] * 2)
print(gen_circuit(w))

[tensor(0.7224, dtype=torch.float64), tensor(0.1574, dtype=torch.float64), tensor(0.2491, dtype=torch.float64), tensor(0.8491, dtype=torch.float64)]


Kokkos::OpenMP::initialize WARNING: OMP_PROC_BIND environment variable not set
  In general, for best performance with OpenMP 4.0 or better set OMP_PROC_BIND=spread and OMP_PLACES=threads
  For best performance with OpenMP 3.1 set OMP_PROC_BIND=true
  For unit testing set OMP_PROC_BIND=false



In [4]:
device = torch.device("cuda")


x = torch.randn(1, 3, 64, 64)
Ɛ = torch.zeros_like(x)

for i in range(x.shape[1]): 
    for j in range(x.shape[2]):
        values = []
        for k in range(0, x.shape[3]):
            if k % config["qubits"] == 0:  
                values = gen_circuit(w)
                random.shuffle(values)
            Ɛ[:, i, j, k] = values[k % config["qubits"]] 

print(Ɛ)
print(Ɛ.shape)


In [ ]:
x = torch.randn(1, 3, 64, 64)

Ɛ = torch.zeros_like(x)

for i in range(x.shape[1]): 
    for j in range(x.shape[2]):  
        for k in range(x.shape[3]):  
            values = gen_circuit(w) 
            Ɛ[:, i, j, k] = values

print(Ɛ)
print(Ɛ.shape)